In [5]:
import findspark
findspark.init()


In [33]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt

In [34]:
spark = SparkSession.builder.master("local[2]").appName("Google playstore app rating prediction").getOrCreate()

In [35]:
from pyspark.ml.regression import LinearRegression

In [36]:
csvdf = spark.read.format("csv").option("header",True).option("inferSchema",True).load("C:\\Users\\Jaisriram\\Desktop\\desktop\\bigdata\\Google-Play-Store-master\\googleplaystore.csv")
csvdf.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [37]:
import pyspark
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape
csvdf.shape()

(10841, 13)

In [38]:
#finding datatypes of features
import pandas as pd
#pd.set_option('max_colwidth', -1) # to prevent truncating of columns in jupyter

def count_column_types(spark_df):
    """Count number of columns per type"""
    return pd.DataFrame(spark_df.dtypes).groupby(1, as_index=False)[0].agg({'count':'count', 'names': lambda x: " | ".join(set(x))}).rename(columns={1:"type"})

In [39]:
count_column_types(csvdf)

,type,count,names
0,string,13,App | Genres | Android Ver | Rating | Price | Current Ver | Type | Content Rating | Installs | Last Updated | Size | Reviews | Category


In [40]:
I have a dataset with missing values , I would like to get the number of missing values for each columns. Following is what I did , I got the number of non missing values. How can I use it to get the number of missing values?

Object `values` not found.


In [ ]:
I have a dataset with missing values , I would like to get the number of missing values for each columns. Following is what I did , I got the number of non missing values. How can I use it to get the number of missing values

In [100]:
#no of missing values
from pyspark.sql.functions import col,sum
csvdf.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in csvdf.columns)).show()

+---+--------+------+-------+----+--------+----+-----+-------+------+------------+-----------+-----------+
|App|Category|Rating|Reviews|Size|Installs|Type|Price|CRating|Genres|Last Updated|Current Ver|Android Ver|
+---+--------+------+-------+----+--------+----+-----+-------+------+------------+-----------+-----------+
|  0|       0|     0|      0|   0|       0|   0|    0|      1|     0|           0|          1|          1|
+---+--------+------+-------+----+--------+----+-----+-------+------+------------+-----------+-----------+



In [58]:
pip install missingno

In [63]:
#import missingno as msno
#msno.bar(csvdf.toPandas())

In [66]:
csvdf = csvdf.withColumnRenamed('Content Rating', 'CRating')
csvdf.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price| CRating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   

In [89]:

csvdf.where(col("Current Ver").isNotNull()).show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price| CRating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AN

In [88]:
csvdf.filter("Rating is null")
csvdf.filter(csvdf.Rating.isNull()).count()

0

In [93]:
from pyspark.sql.functions import lit

csvdf = old_df.withColumn('Rating', lit(None).cast(StringType()))

NameError: name 'old_df' is not defined

In [94]:
from pyspark.sql.functions import udf
val empty = udf(() => null: String)

csvdf.withColumn("Rating", explode(
  when(col("Rating").isNotNull, col("Rating"))
    // If null explode an array<string> with a single null
    .otherwise(array(lit(None).cast("string")))))

SyntaxError: invalid syntax (<ipython-input-94-b7d39fcd8921>, line 2)